# Предобработка

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns; sns.set_style()

Загрузим данные:

In [2]:
!kaggle competitions download -c itmo-andan-competition
!mv itmo-andan-competition.zip data/raw/itmo-andan-competition.zip
!unzip -o data/raw/itmo-andan-competition.zip -d data/raw

100%|██████████████████████████████████████| 9.89M/9.89M [00:00<00:00, 29.1MB/s]
100%|██████████████████████████████████████| 9.89M/9.89M [00:00<00:00, 28.4MB/s]
Archive:  data/raw/itmo-andan-competition.zip
  inflating: data/raw/ads.csv        
  inflating: data/raw/history.csv    
  inflating: data/raw/sample_solution.csv  
  inflating: data/raw/target.csv     
  inflating: data/raw/users.csv      


In [3]:
ads = pd.read_csv('data/raw/ads.csv')

ads.set_index('ad_id', inplace=True)

ads.head()

,cpm,hour_start,hour_end,publishers,audience_size,user_ids
ad_id,,,,,,
0,220.0,1058,1153,"7,17",1906,"12,44,46,50,58,71,93,122,134,143,176,184,187,1..."
1,312.0,1295,1301,"3,18",1380,"29,81,98,102,165,167,195,205,218,231,242,263,3..."
2,70.0,1229,1249,"1,2,3,9,15,21",888,"12,23,25,29,45,85,92,124,156,190,272,334,456,5..."
3,240.0,1295,1377,"1,14",440,"44,122,187,209,242,255,312,345,382,465,513,524..."
4,262.0,752,990,"1,3,7,8",1476,"15,24,30,43,50,53,96,105,159,168,181,190,196,2..."


In [4]:
history = pd.read_csv('data/raw/history.csv')

history.head()

,hour,cpm,publisher,user_id
0,10,30.00,1,15661
1,8,41.26,1,8444
2,7,360.00,1,15821
3,18,370.00,1,21530
4,8,195.00,2,22148


In [5]:
target = pd.read_csv('data/raw/target.csv')

target.set_index('ad_id', inplace=True)

target.head()

,at_least_one
ad_id,
0,0.0430
1,0.0130
2,0.0878
3,0.2295
4,0.3963


In [6]:
users = pd.read_csv('data/raw/users.csv')

users.set_index('user_id', inplace=True)

users.head()

,sex,age,city_id
user_id,,,
0,2,19,0
1,1,0,1
2,2,24,2
3,1,20,3
4,2,29,4


Учиться мы будем на таблице `ads`, давайте добавлять в неё фичи (и убирать ненужные).

## Publishers

Придумаем, что делать с платформами. Понятно, что оставлять их в виде списка бессмысленно, поэтому предложим следующие варианты:
- One-hot encoding (ну почти, ставим `1` тем платформам, на которых мы выпустились)
- Mean encoding (опять же, ну почти - кодируем каким-то числом, которое описывает охват платформы)
- Какая-то другая комбинация признаков

Я не люблю one-hot, он раздувает датасет и вообще с ним много проблем. Попробуем сделать mean encoding.

Поймём, что мы можем узнать из датасета history:
1) Можно записать количество показов конкретной платформы;
2) А можно средний pcm;
3) А можно суммарный pcm!

А почему бы не всё сразу? Во время анализа откинем лишнее!

In [7]:
publisher_counts = history.publisher.value_counts()

publisher_cpm_mean = {}

publisher_cpm_sum = {}

for publisher in publisher_counts.keys():
    cpms = history[history['publisher'] == publisher]['cpm']
    
    publisher_cpm_mean[publisher] = cpms.mean()
    publisher_cpm_sum[publisher] = cpms.sum()
    
def unpack_list_by_dict(lst : str, dct):
    unpacked = map(lambda x : dct[x], map(int, lst.split(',')))
    return np.array(list(unpacked))

In [8]:
ads = ads.assign(
    publisher_counts = ads.publishers.apply(lambda ad : unpack_list_by_dict(ad, publisher_counts)[0]),
    publisher_cpm_mean = ads.publishers.apply(lambda ad : unpack_list_by_dict(ad, publisher_cpm_mean).sum()),
    publisher_cpm_sum = ads.publishers.apply(lambda ad : unpack_list_by_dict(ad, publisher_cpm_sum).sum()),
)

ads.head()

,cpm,hour_start,hour_end,publishers,audience_size,user_ids,publisher_counts,publisher_cpm_mean,publisher_cpm_sum
ad_id,,,,,,,,,
0,220.0,1058,1153,"7,17",1906,"12,44,46,50,58,71,93,122,134,143,176,184,187,1...",66134,419.891791,1.635979e+07
1,312.0,1295,1301,"3,18",1380,"29,81,98,102,165,167,195,205,218,231,242,263,3...",72124,301.112193,1.414966e+07
2,70.0,1229,1249,"1,2,3,9,15,21",888,"12,23,25,29,45,85,92,124,156,190,272,334,456,5...",692535,875.298504,1.936515e+08
3,240.0,1295,1377,"1,14",440,"44,122,187,209,242,255,312,345,382,465,513,524...",692535,374.926350,1.231799e+08
4,262.0,752,990,"1,3,7,8",1476,"15,24,30,43,50,53,96,105,159,168,181,190,196,2...",692535,733.099400,1.536859e+08


Теперь сам список нам не нужен, выбросим его:

In [9]:
ads.drop('publishers', axis=1, inplace=True)

ads.head()

,cpm,hour_start,hour_end,audience_size,user_ids,publisher_counts,publisher_cpm_mean,publisher_cpm_sum
ad_id,,,,,,,,
0,220.0,1058,1153,1906,"12,44,46,50,58,71,93,122,134,143,176,184,187,1...",66134,419.891791,1.635979e+07
1,312.0,1295,1301,1380,"29,81,98,102,165,167,195,205,218,231,242,263,3...",72124,301.112193,1.414966e+07
2,70.0,1229,1249,888,"12,23,25,29,45,85,92,124,156,190,272,334,456,5...",692535,875.298504,1.936515e+08
3,240.0,1295,1377,440,"44,122,187,209,242,255,312,345,382,465,513,524...",692535,374.926350,1.231799e+08
4,262.0,752,990,1476,"15,24,30,43,50,53,96,105,159,168,181,190,196,2...",692535,733.099400,1.536859e+08


## Users

Теперь давайте что-то сделаем с самими пользователями. Про каждго из них мы знаем немного вещей:

1) Пол;
2) Возраст;
3) Город.

Пройдёмся по каждому из этих пунктов.

### Пол

Сначала вспомним, что у нас есть очень забавные люди, которые либо не указали свой пол, либо указали там боевой вертолёт McDonnell Douglas AH-64 Apache. Если их как-то обработать, то можно было бы закодировать пол набора целевых пользователей долей мужчин/женщин. Я вижу два варианта:

1) Если мужчины - `0`, а женщины - `1`, то пусть боевые вертолёты будут обозначаться `0.5`.
2) Можно приписать этих трансформеров к одному из классов. Ну а что? Конституция так говорит.

Мне больше по душе первый вариант. Кажется, это позволяет не сбивать баланс классов.

In [10]:
def user_sex_value(id : str):
    sex = users.iloc[int(id)]['sex']
    if sex == 0:
        return .5
    return sex - 1.

In [11]:
ads = ads.assign(
    user_sexes = ads.user_ids.apply(lambda ids : np.array(list(map(user_sex_value, ids.split(',')))).mean())
)

ads.head()

,cpm,hour_start,hour_end,audience_size,user_ids,publisher_counts,publisher_cpm_mean,publisher_cpm_sum,user_sexes
ad_id,,,,,,,,,
0,220.0,1058,1153,1906,"12,44,46,50,58,71,93,122,134,143,176,184,187,1...",66134,419.891791,1.635979e+07,0.462225
1,312.0,1295,1301,1380,"29,81,98,102,165,167,195,205,218,231,242,263,3...",72124,301.112193,1.414966e+07,0.467391
2,70.0,1229,1249,888,"12,23,25,29,45,85,92,124,156,190,272,334,456,5...",692535,875.298504,1.936515e+08,0.467342
3,240.0,1295,1377,440,"44,122,187,209,242,255,312,345,382,465,513,524...",692535,374.926350,1.231799e+08,0.454545
4,262.0,752,990,1476,"15,24,30,43,50,53,96,105,159,168,181,190,196,2...",692535,733.099400,1.536859e+08,0.387873


### Возраст

На лекции мы уже обрабатывали возраст, и я предлагаю поступить следующим образом: давайте за mean encode-им все выбросы, выбросами будем считать тех людей, у которых возраст больше `80` и тех, у кого стоит `0`. 

Почему это хорошая идея, ведь на распределении возрастов вылетит огромный выброс на среднем возрасте? Потому что на самом деле мы не будем смотреть на распределение, а снова возьмём среднее абсолютно внаглую.

In [12]:
MEAN_AGE = users[(users['age'] != 0) & (users['age'] < 80)]['age'].mean()

def user_age_value(id : str):
    age = users.iloc[int(id)]['age']
    if age == 0 or age > 80:
        return MEAN_AGE
    return float(age)

In [13]:
ads = ads.assign(
    user_ages = ads.user_ids.apply(lambda ids : np.array(list(map(user_age_value, ids.split(',')))).mean())
)

ads.head()

,cpm,hour_start,hour_end,audience_size,user_ids,publisher_counts,publisher_cpm_mean,publisher_cpm_sum,user_sexes,user_ages
ad_id,,,,,,,,,,
0,220.0,1058,1153,1906,"12,44,46,50,58,71,93,122,134,143,176,184,187,1...",66134,419.891791,1.635979e+07,0.462225,29.165062
1,312.0,1295,1301,1380,"29,81,98,102,165,167,195,205,218,231,242,263,3...",72124,301.112193,1.414966e+07,0.467391,28.012621
2,70.0,1229,1249,888,"12,23,25,29,45,85,92,124,156,190,272,334,456,5...",692535,875.298504,1.936515e+08,0.467342,29.823836
3,240.0,1295,1377,440,"44,122,187,209,242,255,312,345,382,465,513,524...",692535,374.926350,1.231799e+08,0.454545,28.669776
4,262.0,752,990,1476,"15,24,30,43,50,53,96,105,159,168,181,190,196,2...",692535,733.099400,1.536859e+08,0.387873,44.760840


### Города

С городами ещё сложнее: в анализе на лекции, как я считаю, была допущена ошибка - `город 0` на самом деле обозначает *не обозначенный город*, а города 3 и 7 это Москва и Петербург. 

Рассмотрим разные подходы:

1) One-hot - вообще очень плохо, у нас куча городов;
2) Какой-то max encoding (aka кодируем городом, в котором больше всего таргетов) - не очень репрезентативно;

Выходит не очень. 

Есть идея - давайте возьмём несколько самых больших городов и объединим все остальные в бин "мелкие города". Тогда можно будет сделать one-hot. 

**НО!** Я не хочу.

Всё, с юзерами разобрались, выкидываем столбец со списком идентификаторов:

In [14]:
ads.drop('user_ids', axis=1, inplace=True)

ads.head()

,cpm,hour_start,hour_end,audience_size,publisher_counts,publisher_cpm_mean,publisher_cpm_sum,user_sexes,user_ages
ad_id,,,,,,,,,
0,220.0,1058,1153,1906,66134,419.891791,1.635979e+07,0.462225,29.165062
1,312.0,1295,1301,1380,72124,301.112193,1.414966e+07,0.467391,28.012621
2,70.0,1229,1249,888,692535,875.298504,1.936515e+08,0.467342,29.823836
3,240.0,1295,1377,440,692535,374.926350,1.231799e+08,0.454545,28.669776
4,262.0,752,990,1476,692535,733.099400,1.536859e+08,0.387873,44.760840


## Hour (start/end)

Не будем делать ничего особенно интеллектуального - поступим так же, как на лекции, просто сохранив длительность.

In [15]:
ads = ads.assign(duration=ads.hour_end - ads.hour_start).drop(['hour_start', 'hour_end'], axis=1)

ads.head()

,cpm,audience_size,publisher_counts,publisher_cpm_mean,publisher_cpm_sum,user_sexes,user_ages,duration
ad_id,,,,,,,,
0,220.0,1906,66134,419.891791,1.635979e+07,0.462225,29.165062,95
1,312.0,1380,72124,301.112193,1.414966e+07,0.467391,28.012621,6
2,70.0,888,692535,875.298504,1.936515e+08,0.467342,29.823836,20
3,240.0,440,692535,374.926350,1.231799e+08,0.454545,28.669776,82
4,262.0,1476,692535,733.099400,1.536859e+08,0.387873,44.760840,238


## Engineering

Теперь давайте понаделаем фичей, которые хоть сколько-то осмысленные, из существующих:

`duration * cpm` выглядит как разумная фича, как и `audience_size * duration`. Вообще, украдём идеи с лекции:

In [16]:
ads = ads.assign(
    audience_x_duration = ads.duration * ads.audience_size,
    duration_x_cpm = ads.cpm * ads.duration
)

## Split'n'Save

Разобьём данные на трейн и тест и сохраним:

In [17]:
ads_train = ads[ads.index < 700]

ads_train

,cpm,audience_size,publisher_counts,publisher_cpm_mean,publisher_cpm_sum,user_sexes,user_ages,duration,audience_x_duration,duration_x_cpm
ad_id,,,,,,,,,,
0,220.0,1906,66134,419.891791,1.635979e+07,0.462225,29.165062,95,181070,20900.0
1,312.0,1380,72124,301.112193,1.414966e+07,0.467391,28.012621,6,8280,1872.0
2,70.0,888,692535,875.298504,1.936515e+08,0.467342,29.823836,20,17760,1400.0
3,240.0,440,692535,374.926350,1.231799e+08,0.454545,28.669776,82,36080,19680.0
4,262.0,1476,692535,733.099400,1.536859e+08,0.387873,44.760840,238,351288,62356.0
...,...,...,...,...,...,...,...,...,...,...
695,40.0,2320,692535,803.729453,1.387341e+08,0.507328,26.848894,451,1046320,18040.0
696,291.0,1260,692535,717.335505,1.948134e+08,0.465079,14.430159,18,22680,5238.0
697,130.0,2357,692535,760.099078,1.930918e+08,0.528426,27.022910,200,471400,26000.0


In [18]:
ads_test = ads[ads.index >= 700]

ads_test

,cpm,audience_size,publisher_counts,publisher_cpm_mean,publisher_cpm_sum,user_sexes,user_ages,duration,audience_x_duration,duration_x_cpm
ad_id,,,,,,,,,,
700,330.0,344,273037,681.593773,7.132538e+07,0.498547,28.629471,5,1720,1650.0
701,79.0,1960,66134,547.040584,1.639654e+07,0.492602,29.790624,151,295960,11929.0
702,130.0,2400,692535,430.153485,1.240588e+08,0.466458,29.371801,5,12000,650.0
703,134.0,1960,273037,853.577041,7.246230e+07,0.477806,29.454107,3,5880,402.0
704,50.0,1428,692535,640.112564,1.785426e+08,0.410014,32.793019,30,42840,1500.0
...,...,...,...,...,...,...,...,...,...,...
1003,127.0,368,273037,808.500795,7.206125e+07,0.456522,29.625482,59,21712,7493.0
1004,90.0,484,273037,249.245734,5.546368e+07,0.432851,29.922937,4,1936,360.0
1005,122.0,704,273037,387.035196,5.679894e+07,0.482955,27.671191,5,3520,610.0


Запишем процессированные данные в файл:

In [19]:
ads_train.to_csv('data/processed/train.csv')
ads_test.to_csv('data/processed/test.csv')
target.to_csv('data/processed/labels.csv')

That's all for preprocessing, folks!